# TextRank



## Библиотеки

In [ ]:
!pip install datasets

     |████████████████████████████████| 325 kB 10.0 MB/s 
     |████████████████████████████████| 1.1 MB 47.9 MB/s 
     |████████████████████████████████| 136 kB 57.9 MB/s 
     |████████████████████████████████| 212 kB 45.4 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 127 kB 41.6 MB/s 
     |████████████████████████████████| 144 kB 43.3 MB/s 
     |████████████████████████████████| 271 kB 49.2 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install transformers sentencepiece

     |████████████████████████████████| 4.0 MB 24.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.5 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 895 kB 58.0 MB/s 
     |████████████████████████████████| 6.6 MB 42.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Датасет

In [ ]:
from datasets import load_dataset
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")['test']


No config specified, defaulting to: gazeta_dataset/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta_dataset downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta_dataset/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
text = dataset[1]['text']

## Код

In [ ]:
import nltk
nltk.download('punkt')
from itertools import combinations
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
import networkx as nx


def similarity(s1, s2):
    if not len(s1) or not len(s2):
        return 0.0

    # В качестве меры похожести возьмем отношение количества одинаковых слов в предложениях
    # к суммарной длине предложений. Коэффициент Сёренсена.
    return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))


def textrank(text):
    # Разбиваем текст на предложения с помощью токенизатора из nltk.
    sentences = sent_tokenize(text)

    # Поделим эти предложения на слова регуляркой, выбрасывая пробелы и знаки препинания.
    tokenizer = RegexpTokenizer(r'\w+')

    # Все слова обработаем стеммером, который обрезает суффиксы и окончания, чтобы 
    # одно и то же слово в разных формах выглядело одинаково.
    lmtzr = RussianStemmer()

    # Выкинем повторяющиеся слова из предложений. После этого у нас есть список предложений, 
    # которые представлены в виде множества слов.
    words = [set(lmtzr.stem(word) for word in tokenizer.tokenize(sentence.lower()))
             for sentence in sentences]
    
    # Для каждой пары предложений вычислим их похожесть
    pairs = combinations(range(len(sentences)), 2)
    scores = [(i, j, similarity(words[i], words[j])) for i, j in pairs]

    # Уберем пары, у которых нет ничего общего (похожесть равна нулю).
    scores = filter(lambda x: x[2], scores)
	
    # Создадим граф. Вершинами в нем являются предложения (если точнее, то номера предложений в тексте), 
    # а ребра между ними имеют вес, равный похожести вершин, которым оно инцидентно.
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Посчитаем пейджранки в этом графе.
    pr = nx.pagerank(g)
	
    # Результатом будет список предложений, отсортированный по их пейджранку.
    return sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr), key=lambda x: pr[x[0]], reverse=True)


# Вывод топ-N предложений
def extract(text, n=5):
    tr = textrank(text)
    top_n = sorted(tr[:n])
    return ' '.join(x[2] for x in top_n)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Вывод

In [ ]:
display(text)

'Около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре Бурятии. В Кремле выступил Бурятский государственный академический театр оперы и балета, Национальный цирк, Бурятский национальный театр песни и танца «Байкал», ставший победителем шоу «Танцуют все!» на телеканале «Россия», а также другие профессиональные и самодеятельные коллективы региона. Более 300 артистов из одного региона на главной сцене страны - похоже это рекорд России. Зрителям рассказали, что Республике Бурятия, чье население составляет 1 миллион человек, сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков. И все они были представлены в Москве. Как писали после шоу зрители в соцсетях: «А мы думали, что в Бурятии только буряты живут…». Для неподготовленного зрителя это вообще были вечера открытий. Например, когда еще в Кремлевском дворце выстраивалась очередь из желающих попасть на прием к врачам-пульсологам и ламам-астр

In [ ]:
display(extract(text, 5))

'Около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре Бурятии. И все они были представлены в Москве. Как писали после шоу зрители в соцсетях: «А мы думали, что в Бурятии только буряты живут…». А между тем буквально в паре метров можно было увидеть выставку Национального музея Бурятии о культуре и быте бурятского народа. Сам же Цыденов в своем выступлении посетовал на то, что в России все же мало что знают о Бурятии и ее традициях, но был оптимистичен: «Республика Бурятия богата своей историей, своими традициями.'